# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [7]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))

# Display the first few rows of the dataframe
fires_dt.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
1,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0
2,7,4,oct,tue,90.6,35.4,669.1,6.7,18,33,0.9,0,0
3,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0
4,8,6,mar,fri,91.7,33.3,77.5,9,8.3,97,4,0.2,0


# Get X and Y

Create the features data frame and target data.

In [8]:
# Separate features and target variable
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Define the ColumnTransformer for preproc1
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Apply log transformation to certain numeric features
log_transformer = Pipeline(steps=[
    ('log', FunctionTransformer(np.log1p, validate=True)),
    ('scaler', StandardScaler())
])




### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [10]:
# Define the ColumnTransformer for preproc2
preproc2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('log', log_transformer, ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [11]:
# Define the baseline regressor
baseline_regressor = LinearRegression()

# Define the advanced regressor
advanced_regressor = RandomForestRegressor(random_state=42)

# Define the pipelines
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])

pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])

pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])

pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])




# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [12]:
# Define the parameter grids
param_grid_A = {'regressor__fit_intercept': [True, False]}
param_grid_B = {'regressor__fit_intercept': [True, False]}
param_grid_C = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}
param_grid_D = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}

# Define GridSearchCV for each pipeline
grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error')
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error')
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error')
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error')

# Fit the grids
grid_A.fit(X, y)
grid_B.fit(X, y)
grid_C.fit(X, y)
grid_D.fit(X, y)


c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users

ValueError: could not convert string to float: 'X'

# Evaluate

+ Which model has the best performance?

In [ ]:
# Get the best models and their performance
best_model_A = grid_A.best_estimator_
best_model_B = grid_B.best_estimator_
best_model_C = grid_C.best_estimator_
best_model_D = grid_D.best_estimator_

# Evaluate performance
performance_A = -grid_A.best_score_
performance_B = -grid_B.best_score_
performance_C = -grid_C.best_score_
performance_D = -grid_D.best_score_

# Find the best model
performances = {
    'Pipeline A': performance_A,
    'Pipeline B': performance_B,
    'Pipeline C': performance_C,
    'Pipeline D': performance_D
}
best_pipeline = min(performances, key=performances.get)
best_model = eval(f'best_model_{best_pipeline[-1]}')

print(f"Best pipeline: {best_pipeline}")
print(f"Best model performance (MSE): {performances[best_pipeline]}")


# Export

+ Save the best performing model to a pickle file.

In [ ]:
import pickle

# Save the best model
with open('/mnt/data/best_forestfire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap

# Use SHAP values for explanations
explainer = shap.Explainer(best_model['regressor'])
shap_values = explainer(best_model['preprocessing'].transform(X))

# Select an observation for local explanation
observation = X.iloc[0]
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], observation)

# Global explanation
shap.summary_plot(shap_values, X)


*(Answer here.)*

In [19]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# Load the data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area'
]
fires_dt = pd.read_csv('/.../.../05_src/data/fires/forestfires.csv', header=None, names=columns)

# Separate features and target variable
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']

# Define the numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Preprocessor 1: Standard scaling for numeric features, one-hot encoding for categorical features
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Preprocessor 2: Apply log transformation to certain numeric features and standard scaling for others
log_transformer = Pipeline(steps=[
    ('log', FunctionTransformer(np.log1p, validate=True)),
    ('scaler', StandardScaler())
])

preproc2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [col for col in numeric_features if col not in ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']]),
        ('log', log_transformer, ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Define the baseline regressor
baseline_regressor = LinearRegression()

# Define the advanced regressor
advanced_regressor = RandomForestRegressor(random_state=42)

# Define the pipelines
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])

pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])

pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])

pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])

# Define the parameter grids
param_grid_A = {'regressor__fit_intercept': [True, False]}
param_grid_B = {'regressor__fit_intercept': [True, False]}
param_grid_C = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}
param_grid_D = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}

# Define GridSearchCV for each pipeline
grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error')
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error')
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error')
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error')

# Fit the grids
grid_A.fit(X, y)
grid_B.fit(X, y)
grid_C.fit(X, y)
grid_D.fit(X, y)

# Get the best models and their performance
best_model_A = grid_A.best_estimator_
best_model_B = grid_B.best_estimator_
best_model_C = grid_C.best_estimator_
best_model_D = grid_D.best_estimator_

# Evaluate performance
performance_A = -grid_A.best_score_
performance_B = -grid_B.best_score_
performance_C = -grid_C.best_score_
performance_D = -grid_D.best_score_

# Find the best model
performances = {
    'Pipeline A': performance_A,
    'Pipeline B': performance_B,
    'Pipeline C': performance_C,
    'Pipeline D': performance_D
}
best_pipeline = min(performances, key=performances.get)
best_model = eval(f'best_model_{best_pipeline[-1]}')

print(f"Best pipeline: {best_pipeline}")
print(f"Best model performance (MSE): {performances[best_pipeline]}")

import pickle

# Save the best model
with open('/mnt/data/best_forestfire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

import shap

# Use SHAP values for explanations
explainer = shap.Explainer(best_model.named_steps['regressor'], best_model.named_steps['preprocessing'].transform(X))
shap_values = explainer.shap_values(best_model.named_steps['preprocessing'].transform(X))

# Select an observation for local explanation
observation = X.iloc[0]
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], observation)

# Global explanation
shap.summary_plot(shap_values, X)


FileNotFoundError: [Errno 2] No such file or directory: '/.../.../05_src/data/fires/forestfires.csv'

In [21]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# Load the data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area'
]
fires_dt = pd.read_csv('../../05_src/data/fires/forestfires.csv', header=None, names=columns)

# Separate features and target variable
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']

# Define the numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Define the ColumnTransformer for preproc1
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Apply log transformation to certain numeric features
log_transformer = Pipeline(steps=[
    ('log', FunctionTransformer(np.log1p, validate=True)),
    ('scaler', StandardScaler())
])

# Define the ColumnTransformer for preproc2
preproc2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [col for col in numeric_features if col not in ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']]),
        ('log', log_transformer, ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Define the baseline regressor
baseline_regressor = LinearRegression()

# Define the advanced regressor
advanced_regressor = RandomForestRegressor(random_state=42)

# Define the pipelines
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])

pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])

pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])

pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])

# Define the parameter grids
param_grid_A = {'regressor__fit_intercept': [True, False]}
param_grid_B = {'regressor__fit_intercept': [True, False]}
param_grid_C = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}
param_grid_D = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}

# Define GridSearchCV for each pipeline
grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error')
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error')
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error')
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error')

# Fit the grids
grid_A.fit(X, y)
grid_B.fit(X, y)
grid_C.fit(X, y)
grid_D.fit(X, y)

# Get the best models and their performance
best_model_A = grid_A.best_estimator_
best_model_B = grid_B.best_estimator_
best_model_C = grid_C.best_estimator_
best_model_D = grid_D.best_estimator_

# Evaluate performance
performance_A = -grid_A.best_score_
performance_B = -grid_B.best_score_
performance_C = -grid_C.best_score_
performance_D = -grid_D.best_score_

# Find the best model
performances = {
    'Pipeline A': performance_A,
    'Pipeline B': performance_B,
    'Pipeline C': performance_C,
    'Pipeline D': performance_D
}
best_pipeline = min(performances, key=performances.get)
best_model = eval(f'best_model_{best_pipeline[-1]}')

print(f"Best pipeline: {best_pipeline}")
print(f"Best model performance (MSE): {performances[best_pipeline]}")

# Save the best model
import pickle
with open('/mnt/data/best_forestfire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Use SHAP values for explanations
import shap
explainer = shap.Explainer(best_model.named_steps['regressor'], best_model.named_steps['preprocessing'].transform(X))
shap_values = explainer.shap_values(best_model.named_steps['preprocessing'].transform(X))

# Select an observation for local explanation
observation = X.iloc[0]
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], observation)

# Global explanation
shap.summary_plot(shap_values, X)


c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "c:\Users\rijan\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users

ValueError: could not convert string to float: 'X'

In [27]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import shap

# Load the data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area'
]
fires_dt = pd.read_csv('../../05_src/data/fires/forestfires.csv', header=None, names=columns)

# Verify and convert numeric columns to numeric data types explicitly
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

for col in numeric_features:
    fires_dt[col] = pd.to_numeric(fires_dt[col], errors='coerce')

# Convert the target variable to numeric
fires_dt['area'] = pd.to_numeric(fires_dt['area'], errors='coerce')

# Drop rows where target variable is NaN
fires_dt = fires_dt.dropna(subset=['area'])

# Separate features and target variable
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']

# Define preprocessors
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

log_transformer = Pipeline(steps=[
    ('log', FunctionTransformer(np.log1p, validate=True)),
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preproc1 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Drop any columns not specified in transformers
)

preproc2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, [col for col in numeric_features if col not in ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']]),
        ('log', log_transformer, ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'wind']),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Drop any columns not specified in transformers
)

# Define model pipelines
baseline_regressor = LinearRegression()
advanced_regressor = RandomForestRegressor(random_state=42)

pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline_regressor)
])

pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', baseline_regressor)
])

pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced_regressor)
])

pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', advanced_regressor)
])

# Define hyperparameter grids and perform GridSearch
param_grid_A = {'regressor__fit_intercept': [True, False]}
param_grid_B = {'regressor__fit_intercept': [True, False]}
param_grid_C = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}
param_grid_D = {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [None, 10, 20]}

grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error')
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error')
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error')
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error')

# Fit the grids
grid_A.fit(X, y)
grid_B.fit(X, y)
grid_C.fit(X, y)
grid_D.fit(X, y)

# Get the best models and their performance
best_model_A = grid_A.best_estimator_
best_model_B = grid_B.best_estimator_
best_model_C = grid_C.best_estimator_
best_model_D = grid_D.best_estimator_

performance_A = -grid_A.best_score_
performance_B = -grid_B.best_score_
performance_C = -grid_C.best_score_
performance_D = -grid_D.best_score_

performances = {
    'Pipeline A': performance_A,
    'Pipeline B': performance_B,
    'Pipeline C': performance_C,
    'Pipeline D': performance_D
}
best_pipeline = min(performances, key=performances.get)
best_model = eval(f'best_model_{best_pipeline[-1]}')

print(f"Best pipeline: {best_pipeline}")
print(f"Best model performance (MSE): {performances[best_pipeline]}")

# Save the best model
with open('/mnt/data/best_forestfire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# SHAP explanation for the best model
explainer = shap.Explainer(best_model.named_steps['regressor'], best_model.named_steps['preprocessing'].transform(X))
shap_values = explainer.shap_values(best_model.named_steps['preprocessing'].transform(X))

# Select an observation for local explanation
observation = X.iloc[0]
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], observation)

# Global explanation
shap.summary_plot(shap_values, X)


Best pipeline: Pipeline B
Best model performance (MSE): 4427.718913195339


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/best_forestfire_model.pkl'

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.